# Natural Language Patient Finder

Identify patients stored in a FHIR server that match a natural language fragment.

In [ ]:
import requests
import json

# Set target services
Services usually share a namespace. Specify namespace to be used.

In [ ]:
ingress_subdomain = ""

namespace = "hackathon" # input("Enter a service namespace: ")

#Learned Intent endpoint
intentURL = "https://" + namespace  + "-learned-intent." + ingress_subdomain + "/?text="

#Key Concept Extractor endpoint
extractConceptsURL = "https://" + namespace + "-key-concept-extractor." + ingress_subdomain

#Generate CQL endpoint
generateCQLURL = "https://" + namespace + "-cql-generator." + ingress_subdomain + "/generate"

#libraries
cohortURL = "https://hackathon-ingestion-cohort-service." + ingress_subdomain + "/cohort-service/libraries"
#/libraries/nameoflibrary  (get or delete)
#/libraries (post new library)
#/libraries/nameoflibrary/patients (return list of patients)
#/libraries/nameoflibrary/patientIds (return list of patient ids)

# Provide fragment to be analyzed
Examples:
- Female patients > 18 years old
- Patient has diabetes mellitus
- Hypertriglyceridemia
- nonproliferative diabetic retinopathy
- no gestational diabetes
- Patient is able to return to Johns Hopkins for follow-up appointments.

In [ ]:
fragment = input("Enter a fragment: ")

# Identify Intent of fragment

In [ ]:
url = intentURL + "\"" + fragment + "\""

resp = requests.get(url=url)
intent = resp.content.decode("utf-8")
print("Learned intent is: ", intent)

# Extract key entities for given fragment and intent

In [ ]:
params = {'intent': intent, 'text': fragment}
  
# sending get request and saving the response as response object
concept_set = requests.get(url = extractConceptsURL, params = params).content.decode("utf-8")

parsed = json.loads(concept_set)
print(json.dumps(parsed, indent=4, sort_keys=True))

# (Optional) show SnoMed concept expansion

In [ ]:
snomed_code = input("Enter SnoMed code to expand: ")

snomed_expansion_url = "https://snowstorm-fhir.snomedtools.org/fhir/CodeSystem/$lookup?code=" + snomed_code;

expansion_response = requests.get(url=snomed_expansion_url)

parsed = json.loads( expansion_response.content.decode("utf-8"))
print("\nSnoMed expansion:")
print(json.dumps(parsed, indent=4, sort_keys=True))

# Generate CQL from concepts and identified type

In [ ]:
cql_response = requests.post(url=generateCQLURL, data={'conceptSet': concept_set})

cql = cql_response.content.decode("utf-8")
print("\nCQL Generator produced:\n\n", cql)

# Push CQL to Cohort Service and Run

In [ ]:
library_name = cql.split('"')[1]
print("name:", library_name)

library_version = cql.split("'")[1]
print("version:", library_version)

library_endpoint = library_name + "-" + library_version
print("endpoint extension:", library_endpoint)

#verify no existing CQL exists with this name. Delete if it does.
resp = requests.delete(cohortURL + "/" + library_endpoint)
print(resp.content.decode("utf-8"))

#Push CQL to cohort service
headers = {}
headers["Content-Type"] = "text/plain"
headers["Accept"] = "application/json"

resp = requests.post(url = cohortURL, data = cql, headers = headers)

print("Posting CQL to Cohort Service: " + str(resp))

# Run CQL to find matching patient ID's
run_endpoint = cohortURL + "/" + library_endpoint + "/patientIds"
resp = requests.get(url=run_endpoint)

print("Running CQL on Cohort Service: " + str(resp))

result = resp.content.decode("utf-8")

ids = (eval(result))
print(len(ids), "patients returned")
for id in ids:
    print(id)

# Run CQL on cohort service to identify matching patients

In [ ]:
run_endpoint = cohortURL + "/" + library_endpoint + "/patients"
total_resp = requests.get(url=run_endpoint)

print(json.dumps(total_resp.json(), indent=4, sort_keys=True))